In [1]:
from pathlib import Path
import os
import sys
import gc
import re
import shutil
import json
import math
import jinja2
from collections import defaultdict
import numpy as np
import pandas as pd
import bitsandbytes
import accelerate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from scml import pandasx as pdx
import lalaes2 as mylib
print(f"accelerate={accelerate.__version__}, bitsandbytes={bitsandbytes.__version__}")

accelerate=0.30.1, bitsandbytes=0.43.1


In [2]:
class ModelConf(NamedTuple):
    directory: Path
    model_max_length: int
    char_limit: int
    load_in_dtype: str = "auto"


#input_filename = "persuade_02.parquet"
input_filename = "comp_02.parquet"
template_version = 3
llm_key = "Qwen2-1.5B-Instruct"
llm_map: Dict[str, ModelConf] = {
    "gemma-1.1-2b-it": ModelConf(
        directory=Path("huggingface/google/gemma-1.1-2b-it"),
        model_max_length=8192,
        char_limit=3000,
        load_in_dtype="fp16",
    ),
    "Qwen2-1.5B-Instruct": ModelConf(
        directory=Path("huggingface/Qwen/Qwen2-1.5B-Instruct"),
        model_max_length=8192,
        char_limit=3000,
    ),
}
mc = llm_map[llm_key]
critique_col = f"ctq_{template_version}_{llm_key}"
environment = jinja2.Environment()
basic = mylib.BasicPreprocessor()

In [3]:
v1_template = environment.from_string(
"""You are a teacher grading a student's essay. Assign a score on a scale of 1 to 6 based on the following rubric. A score of 6 indicates clear and consistent mastery with minor errors, showcasing insightful development of a viewpoint, outstanding critical thinking, strong use of appropriate examples and evidence, well-organized structure with coherence and smooth progression of ideas, skillful language use with varied vocabulary and sentence structure, and minimal errors in grammar, usage, and mechanics. Scores decrease with occasional errors (score of 5), lapses in quality (score of 4), weaknesses in critical thinking and organization (score of 3), serious flaws in viewpoint and coherence (score of 2), and fundamental errors hindering meaning (score of 1).
In your response, output only the score without explanation.
[ESSAY] {{ essay }} [/ESSAY]
"""
)
v2_template = environment.from_string(
"""You are a teacher grading a student's essay. Assign a score on a scale of 1 to 6 based on the following rubric. A score of 6 indicates clear and consistent mastery with minor errors, showcasing insightful development of a viewpoint, outstanding critical thinking, strong use of appropriate examples and evidence, well-organized structure with coherence and smooth progression of ideas, skillful language use with varied vocabulary and sentence structure, and minimal errors in grammar, usage, and mechanics. Scores decrease with occasional errors (score of 5), lapses in quality (score of 4), weaknesses in critical thinking and organization (score of 3), serious flaws in viewpoint and coherence (score of 2), and fundamental errors hindering meaning (score of 1).
Provide constructive feedback to improve the essay.
[ESSAY] {{ essay }} [/ESSAY]
"""
)
v3_template = environment.from_string(
"""You are a teacher grading a student's essay. Assign a score on a scale of 1 to 6 based on the following rubric. A score of 6 indicates clear and consistent mastery with minor errors, showcasing insightful development of a viewpoint, outstanding critical thinking, strong use of appropriate examples and evidence, well-organized structure with coherence and smooth progression of ideas, skillful language use with varied vocabulary and sentence structure, and minimal errors in grammar, usage, and mechanics. Scores decrease with occasional errors (score of 5), lapses in quality (score of 4), weaknesses in critical thinking and organization (score of 3), serious flaws in viewpoint and coherence (score of 2), and fundamental errors hindering meaning (score of 1).
In your response, state the score and areas for improvement.
[ESSAY] {{ essay }} [/ESSAY]
"""
)
template = v1_template
max_new_tokens = 16
if template_version==2:
    template = v2_template
    max_new_tokens = 64
if template_version==3:
    template = v3_template
    max_new_tokens = 64

In [4]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [5]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:1")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [6]:
df = pd.read_parquet(Path(f"input/{input_filename}"))
#df = df.sample(frac=0.001)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   essay_id   17307 non-null  object
 1   score      17307 non-null  int8  
 2   topic      17307 non-null  object
 3   full_text  17307 non-null  object
dtypes: int8(1), object(3)
memory usage: 422.7+ KB


# LLM-as-a-Judge

LLM takes on the role of a teacher to generate essay critique.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(mc.directory, model_max_length=mc.model_max_length)
tokenizer.pad_token = tokenizer.eos_token
print(f"""{tokenizer}
model_input_names={tokenizer.model_input_names}
pad_token_id={tokenizer.pad_token_id}
""")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2TokenizerFast(name_or_path='huggingface/Qwen/Qwen2-1.5B-Instruct', vocab_size=151643, model_max_length=8192, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|im_end|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', 'attention_mask']
pad_token_id=151645



In [8]:
%%time
print(f"Load in {mc.load_in_dtype}")
if mc.load_in_dtype=="fp16":
    model = AutoModelForCausalLM.from_pretrained(
        mc.directory,
        device_map="auto",
        torch_dtype=torch.float16,
        revision="float16",
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        mc.directory,
        torch_dtype="auto",
        device_map="auto"
    )

Load in auto
CPU times: user 4.41 s, sys: 3.25 s, total: 7.65 s
Wall time: 1min 28s


In [9]:
print(model)
print(model.config)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear

In [10]:
def critique(row) -> str:
    """Inference method and response extraction based on https://huggingface.co/Qwen/Qwen2-7B-Instruct"""
    prompt = template.render(essay=str(row["full_text"])[:mc.char_limit])
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer(text, truncation=True, return_tensors="pt").to(device)
    model_outputs = model.generate(
        **model_inputs, 
        max_new_tokens=max_new_tokens,
        do_sample=False,
        #temperature=0.01,
        #top_p=0.95,
        #top_k=40,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, model_outputs)]
    res = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    res = basic(res)
    if len(res)==1 and res.isdigit():
        res = f"SCORE OF {res}"
    return res


df[critique_col] = df.progress_apply(critique, axis=1)

  0%|                                                                                                                                                             | 0/17307 [00:00<?, ?it/s]/home/pankun/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pankun/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/pankun/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag 

In [11]:
pdx.value_counts(df[critique_col]).head(30)

,count,percent
ctq_3_Qwen2-1.5B-Instruct,,
"Score: 2 Areas for Improvement: - Lack of supporting evidence and examples to back up claims made in the essay. - Poor organization and flow of ideas, making it difficult to follow the argument presented. - Use of vague and unclear language, which makes it hard to understand the writer's point of view. -",41,0.002369
"Score: 3 Areas for Improvement: - Lack of supporting evidence and examples to back up claims made in the essay. - Poor organization and flow of ideas, making it difficult to follow the argument presented. - Use of vague and unclear language, which makes it hard to understand the writer's point of view. -",23,0.001329
"The essay demonstrates clear and consistent mastery with minor errors. The writer showcases insightful development of a viewpoint, outstanding critical thinking, strong use of appropriate examples and evidence, well-organized structure with coherence and smooth progression of ideas, skillful language use with varied vocabulary and sentence structure, and minimal errors in grammar, usage, and",21,0.001213
"Score: 3 Areas for Improvement: - Lack of supporting evidence and examples to back up claims made in the essay. - Poor organization and flow of ideas, making it difficult to follow the argument presented. - Grammar and spelling errors, which detract from the overall quality of the writing. - Lack of clarity in",18,0.001040
Score: 5 Areas for Improvement: - The essay could benefit from a clearer introduction and conclusion. - More specific examples and evidence could strengthen the argument presented in the essay. - The essay could benefit from a stronger thesis statement that clearly outlines the main point being made. - The essay could benefit from a more,9,0.000520
"Score: 2 Areas for Improvement: - Lack of supporting evidence and examples to back up claims made in the essay. - Poor organization and flow of ideas, making it difficult to follow the argument presented. - Use of vague and unclear language, which makes it hard to understand the writer's perspective. - Lack of",8,0.000462
"Score: 3 Areas for Improvement: - Lack of coherence and flow in the essay. - Inconsistent use of language and sentence structure. - Errors in grammar, usage, and mechanics. - Weakness in critical thinking and organization. - Flaws in viewpoint and coherence. - Fundamental errors hindering meaning. Overall",7,0.000404
"Score: 4 Areas for Improvement: - Lack of supporting evidence and examples to back up claims made in the essay. - Poor organization and flow of ideas, making it difficult to follow the argument presented. - Grammar and spelling errors, which detract from the overall quality of the writing. - Lack of clarity in",6,0.000347
"Score: 5 Areas for Improvement: - The essay lacks a clear thesis statement. A thesis statement should clearly define the main argument or point of the essay. - There are several grammatical errors throughout the essay, such as missing articles, incorrect verb tense, and run-on sentences. - The essay does not provide",5,0.000289


# Review Data

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   essay_id                   17307 non-null  object
 1   score                      17307 non-null  int8  
 2   topic                      17307 non-null  object
 3   full_text                  17307 non-null  object
 4   ctq_3_Qwen2-1.5B-Instruct  17307 non-null  object
dtypes: int8(1), object(4)
memory usage: 557.9+ KB


In [13]:
cols = [critique_col, "score", "full_text"]
df[cols].head(10)

,ctq_3_Qwen2-1.5B-Instruct,score,full_text
0,Score: 5 Areas for Improvement: - Grammar and Usage: There are a few instances of incorrect grammar and poor sentence structure throughout the essay. These should be corrected to improve clarity and flow. - Organization: The essay could benefit from a more organized structure with clearer transitions between paragraphs. This would make the argument more,3,"Many people have car where they live. The thing they don't know is that when you use a car alot of thing can happen like you can get in accidet or the smoke that the car has is bad to breath on if someone is walk but in VAUBAN,Germany they dont have that proble because 70 percent of vauban's families do not own cars,and 57 percent sold a car to move there. Street parkig ,driveways and home garages are forbidden on the outskirts of freiburd that near the French and Swiss borders. You probaly won't see a car in Vauban's streets because they are completely ""car free"" but If some that lives in VAUBAN that owns a car ownership is allowed,but there are only two places that you can park a large garages at the edge of the development,where a car owner buys a space but it not cheap to buy one they sell the space for you car for $40,000 along with a home. The vauban people completed this in 2006 ,they said that this an example of a growing trend in Europe,The untile states and some where else are suburban life from auto use this is called ""smart planning."" The current efforts to drastically reduce greenhouse gas emissions from tailes the passengee cars are responsible for 12 percent of greenhouse gas emissions in Europe and up to 50 percent in some car intensive in the United States. I honeslty think that good idea that they did that is Vaudan because that makes cities denser and better for walking and in VAUBAN there are 5,500 residents within a rectangular square mile. In the artical David Gold berg said that ""All of our development since World war 2 has been centered on the cars,and that will have to change"" and i think that was very true what David Gold said because alot thing we need cars to do we can go anyway were with out cars beacuse some people are a very lazy to walk to place thats why they alot of people use car and i think that it was a good idea that that they did that in VAUBAN so people can see how we really don't need car to go to place from place because we can walk from were we need to go or we can ride bycles with out the use of a car. It good that they are doing that if you thik about your help the earth in way and thats a very good thing to. In the United states ,the Environmental protection Agency is promoting what is called ""car reduced""communtunties,and the legislators are starting to act,if cautiously. Maany experts expect pubic transport serving suburbs to play a much larger role in a new six years federal transportation bill to approved this year. In previous bill,80 percent of appropriations have by law gone to highways and only 20 percent to other transports. There many good reason why they should do this."
1,"Score: 3 Areas for Improvement: - The essay lacks coherence and flow. The writer jumps from topic to topic without providing a clear connection between them. - The essay contains several factual inaccuracies, such as stating that there is no plant life on Mars when in fact there are some plants growing there. - The",3,"I am a scientist at NASA that is discussing the ""face"" on mars. I will be explaining how the ""face"" is a land form. By sharing my information about this isue i will tell you just that. First off, how could it be a martions drawing. There is no plant life on mars as of rite now that we know of, which means so far as we know it is not possible for any type of life. That explains how it could not be made by martians. Also why and how would a martion build a face so big. It just does not make any since that a martian did this. Next, why it is a landform. There are many landforms tha

In [14]:
df.to_parquet(f"output/{input_filename}", index=False)
assert df.notna().all(axis=None)

In [15]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 14:01:45.407047
